In [15]:
my_matrix = [
    [1, 2, 3, 4, 5],
    [6, 7, 8, 9, 10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, 20],
    [21, 22, 23, 24, 25]
]

def transpose_matrix(matrix):
    #return [row for row in zip(*matrix)]
    #return [list(row) for row in zip(*matrix)]
    #return list(zip(**matrix))
    #return list(zip(*matrix))
    return list(map(list, zip(*matrix)))

print(transpose_matrix(my_matrix))

[[1, 6, 11, 16, 21], [2, 7, 12, 17, 22], [3, 8, 13, 18, 23], [4, 9, 14, 19, 24], [5, 10, 15, 20, 25]]


In [54]:
import numpy as np
from PIL import Image, ImageDraw
import cv2

beer_path = "beermug.jpg"
heart_path = "heart.jpg"

def load_images(path_beer, path_heart):
    beer = cv2.imread(path_beer, 0)
    heart = cv2.imread(path_heart, 0)

    beer = cv2.resize(beer, dsize=(500, 400))
    heart = cv2.resize(heart, dsize=(500, 400))

    beer = ((beer > 128) * 255).astype('uint8')
    heart = ((heart > 128) * 255).astype('uint8')

    return beer, heart

def get_coords(image):
    coords = np.column_stack(np.where(image == 0))
    return [tuple(coord) for coord in coords]

def distance(point_pair):
    (x0, y0), (x1, y1) = point_pair
    return np.sqrt((x1 - x0) ** 2 + (y1 - y0) ** 2)

def mean_distance(start_coords, end_coords):
    assert len(start_coords) == len(end_coords)
    distances = [distance(pair) for pair in zip(start_coords, end_coords)]
    return np.mean(distances)

def make_size_equal(coord_beer, coord_heart):
    flipped = False
    more = len(coord_beer) - len(coord_heart)
    if more < 0:
        coord_heart, coord_beer = coord_beer, coord_heart
        more = -more
        flipped = True
    if more > 0:
        coord_heart = sorted(coord_heart)
        coord_beer = sorted(coord_beer)
        adding = coord_heart[-more:]
        coord_heart += adding
    assert len(coord_beer) == len(coord_heart)
    if flipped:
        return coord_heart, coord_beer
    return coord_beer, coord_heart

def make_inner_frames(start_coords, end_coords, N_STEPS=20):
    paths = []
    for i in range(len(start_coords)):
        x0, y0 = start_coords[i]
        xt, yt = end_coords[i]

        xs = np.linspace(x0, xt, N_STEPS + 1)
        ys = np.linspace(y0, yt, N_STEPS + 1)

        path = np.array((xs, ys))
        paths.append(path)
    paths = np.array(paths)
    return paths

def jobb_also(image_coords, pixel_array):
    total_pixels = 0
    print(image_coords)
    for x, y in zip(image_coords[0], image_coords[1]):
        if x >= 200 and y >= 250 and pixel_array[int(x), int(y)] == 0:
            total_pixels += 1

    return total_pixels


def draw_point_shape(xs, ys):
    image = Image.new("RGB", (500, 400), "black")
    draw = ImageDraw.Draw(image)
    for x, y in list(zip(xs, ys)):
        draw.point((y, x))
    return image

def make_gif(paths, N_STEPS):
    frames = []
    for number in range(3):
        frames.append(draw_point_shape(*paths[:, :, 0].transpose()))
    for number in range(N_STEPS + 1):
        frames.append(draw_point_shape(*paths[:, :, number].transpose()))
    for number in range(3):
        frames.append(draw_point_shape(*paths[:, :, -1].transpose()))
    for number in range(N_STEPS, -1, -1):
        frames.append(draw_point_shape(*paths[:, :, number].transpose()))

    frame_one = frames[0]
    frame_one.save("transformation.gif", format="GIF", append_images=frames,
                   save_all=True, duration=100, loop=0)

In [55]:
def main(path_beer, path_heart):
    beer, heart = load_images(path_beer, path_heart)
    coord_beer = get_coords(beer)
    coord_heart = get_coords(heart)
    coord_beer, coord_heart = make_size_equal(coord_beer, coord_heart)
    avg_distance = mean_distance(coord_beer, coord_heart)
    
    print(f"A söröskorsót és a szívet ábrázoló két kép összepárosított fekete képkockáinak átlagos távolsága: {avg_distance:.2f}")
    
    N_STEPS = 20
    paths = make_inner_frames(coord_beer, coord_heart, N_STEPS)
    
    print("A teljes képsorozat koordinátáit tartalmazó numpy array dimenziói:", paths.shape)
    fourth_image_coords = paths[:, :, 3]
    fourth_image_coords = fourth_image_coords.T
    valasz = jobb_also(fourth_image_coords, beer)
    
    print(f"A képsorozat 4. képének ennyi fekete képpontja esik a vászon jobb alsó negyedébe: {valasz}")
    make_gif(paths, N_STEPS)

if __name__ == '__main__':
    x = main("beermug.jpg", "heart.jpg")

A söröskorsót és a szívet ábrázoló két kép összepárosított fekete képkockáinak átlagos távolsága: 137.13
A teljes képsorozat koordinátáit tartalmazó numpy array dimenziói: (84332, 2, 21)
[[  7.65   7.65   7.65 ... 393.45 393.45 393.45]
 [107.95 110.75 111.75 ... 343.45 344.45 345.45]]
A képsorozat 4. képének ennyi fekete képpontja esik a vászon jobb alsó negyedébe: 17984
